In [0]:
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

int main(int argc, char **argv)
{
 volatile int changed;
 char buffer[64];

 if(argc == 1) {
  errx(1, "argument required\n");
 }

 changed = 0;
 strcpy(buffer, argv[1]);

 if(changed == 0x61626364) {
  printf("PWNED!\n");
 } else {
  printf("Not quite! You got: 0x%08x\n", changed);
 }
}

In [0]:
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

int main(int argc, char **argv)
{
 volatile int changed;
 char buffer[64];
 char *variable;

 variable = getenv("PINEAPPLE_PIZZA");

 if(variable == NULL) {
  errx(1, "please set the PINEAPPLE_PIZZA environment variable\n");
 }

 changed = 0;
 strcpy(buffer, variable);

 if(modified == 0x68616579) {
  printf("PWNED!\n");
 } else {
  printf("Not quite! You got: 0x%08x\n", changed);
 }
}

#BREAK M04R of the things!

Welcome back to destroying more applications - hopefully you have already completed part one and are ready to learn some more about break applications.

Let's look at another simple application to smash. This application takes an arguement from the command line much like our broken down application in the last, only it has a much larger buffer!

### What's Happening

Adapted from our last code base we do almost the same. The only different is where we grab our input from and how we apply it with the strcpy and overflow the buffer.
```
volatile int changed;
char buffer[64];
```
The new code:
```
variable = getenv("PINEAPPLE_PIZZA");
```

The program checks if the variable “variable” has a value or not
```
if(variable == NULL) {
  errx(1, "please set the PINEAPPLE_PIZZA environment variable\n");
 }
```
Again, sets the value of changed to 0
```
changed = 0;
```
Then it copies the value of “variable” into the buffer of `changed` which we intend to to fit, right?
```
strcpy(buffer, variable);
```
After that it checks if we have overflowed the buffer to give the required "password" in Hex.

Once again:

`gcc example4.c example4 -fno-stack-protector`

and:

```
export PINEAPPLE_PIZZA="IsTheBest"
gdb ./example4
(gdb) run
```

Now let's see what is happening. We need to overflow the buffer as previous but using the `environmental variables` we can decode the hex string and reverse it to match the required value to hit the execution point we want to hit in the program using python
```
python3 -c 'bytes.fromhex("68616579").decode("utf-8")'
```
To work out what we need to send to the program to trigger it remeber this value will need to be reversed on to the stack!

## Jumping further down the stack
Now you are armed with a few more ways to attack applications and see how they work and what you are looking for. Some familiar patterns in the exercises we have looked at hold true for other, more professional applications even in modern software. 

You should also see how we are able to push our way to other parts of the stack and rewrite them to cause different behvaiour that should not happen within the application.

In the next lesson we will learn how to smash further into the stack!

Happy Hacking!

Tasks:
- We have explored a number of attack vectors to different problems and broken the application. Let's extend the tool to work with all.

##What is happening:

- creates a variable called `changed` and gives it a buffer of 64 chars.

```
volatile int changed; 
char buffer[64];
```
- Checks if we gave an argument or not. (This is (dcba in hex value if anyone is interested)

- Sets the value of the `changed` variable to 0 then it copies whatever we give it `argv[1]` (our argument) into the buffer of `changed`.
```
changed = 0;
strcpy(buffer, argv[1]);
```
Then it checks if the variable’s value is 0x61626364 abcd in hex (or as far as we are concerned dcba backwards! as we need to push on to the stack in reverse!).
```
if(changed == 0x61626364) {
  printf("PWNED!\n");
 } else {
  printf("Not quite! you got 0x%08x\n", changed);
 }
```
We can see this ourselves if we copy it and compile it. Save the code as `example3.c`
```
gcc example3.c example3 -fno-stack-protector
```
Will quickly compile the code and allow us to run it.
```
./example3
exmaple3: argument required
```

Now it is handy the `gdb` has a nifty way of running python. It is actually embedded into our environment for us. This allows us to extend the `gdb` shell further and help us hack away at our applications. To give you a taste take the following script:
```
# We import a gdb handling module
import gdb

# And create a class for our command that extends the GDB command object (notice the capital 'C' which is part of convention)
class MyFirstGDBCommand(gdb.Command):
    # We need to override the generic __init__ and provide a custom one to give the command a friendly name
    def __init___(self):
        super(MyFirstGDBCommand, self).__init__(
          "my_first_gdb_command",  # Command name 
          gdb.COMMAND_DATA
        )

    # The function that runs when we call
    # our command is the invoke function
    def invoke(self, arg, from_tty):
        # This code is what will run when
        # we call it!
        print("YAY! It worked! 😹")

        # We can call our program(s) with
        # the inferiors method()
        for i in gdb.inferiors():
            print(i)
            print(type(i))
            print(dir(i))
            print(help(i))

MyFirstGDBCommand()
```

Now if we save the above to a file `test_gdb_command.py` next to our binary and execute:
```
$ gdb ./example2
```
we should get out friendly `gdb` prompt and now:
```
(gdb) source test_gdb_command.py
(gdb) my_first_gdb_command
YAY! It worked! 😹

...

(gdb)
```

So you can see how we can script our method.

Tasks: 
- Create a python script to explore the inferiors
- Create a script that will flood the application and work out how much is needed to overflow the buffer
- Make the script push the correct value we want so the registers flood with the value we want

Once you have PWNED! this exercise, let's move on to something a little more challenging again!

## Environment Variables

The next piece of code uses another injection mechanism that is often overlooked. The Environment Variables. To work with these we need to export some values before we run the application. This can be done with:
```
export PINEAPPLE_PIZZA='IsTheBest'
```

Let's take a look at the code for this challenge: